In [1]:
import gzip
from collections import defaultdict
import scipy
import scipy.optimize
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [2]:
def readJSON(path):
  for l in gzip.open(path, 'rt'):
    d = eval(l)
    u = d['userID']
    try:
      g = d['gameID']
    except Exception as e:
      g = None
    yield u,g,d

In [4]:
# Get dataset
dataset = []

for user,game,d in readJSON("../data/train.json.gz"):
    hours = d['hours_transformed']
    dataset.append([user, game, hours])

In [5]:
len(dataset)

175000

In [6]:
dataset[0]

['u01561183', 'b96045472', 0.37851162325372983]

In [7]:
# Split into train and validation sets
train_data = dataset[:165000]
val_data = dataset[165000:]

#train_data, val_data = train_test_split(dataset, test_size = 0.05714, random_state = 42)

In [8]:
# Utility data structures
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
usersPerItem = defaultdict(set) # U_i from class slides
itemsPerUser = defaultdict(set) # I_u from class slides
users = set() # all users
items = set() # all games

In [9]:
# Fill data structures using training data
for user,game,hours in train_data:
    reviewsPerUser[user].append([game, hours])
    reviewsPerItem[game].append([user, hours])
    usersPerItem[game].add(user)
    itemsPerUser[user].add(game)
    users.add(user)
    items.add(game)

In [10]:
N = len(train_data)
nUsers = len(reviewsPerUser)
nItems = len(reviewsPerItem)

In [11]:
reviewsPerUser['u20664377']

[['b88846011', 5.4495613746132365],
 ['b03263670', 7.4371278442728075],
 ['b12958376', 5.776103988073165],
 ['b31554490', 3.4724877714627436],
 ['b31270908', 2.944858445807539],
 ['b18850161', 0.13750352374993502],
 ['b66396453', 4.77082904603249],
 ['b21012510', 3.584962500721156],
 ['b99652102', 4.638073837180719],
 ['b26091091', 4.465974464504069],
 ['b56676864', 5.60584986719498],
 ['b99449444', 10.449148645375436]]

In [12]:
reviewsPerItem['b47869546']

[['u47989228', 4.638073837180719],
 ['u77294405', 4.329123596291566],
 ['u77838017', 6.800899899920305],
 ['u77171889', 3.786596361890807],
 ['u16327389', 4.542258049766918],
 ['u36776744', 7.047669251391315],
 ['u46035835', 3.5109619192773796],
 ['u73060782', 5.872828759534886],
 ['u66033184', 6.795715006501729],
 ['u45573247', 7.861086905995394],
 ['u39232843', 2.7224660244710908],
 ['u67100230', 1.8875252707415875],
 ['u20814470', 4.82273014794452],
 ['u09579021', 2.070389327891398],
 ['u69984876', 6.619119511453218],
 ['u54196851', 4.5668151540108965],
 ['u73195763', 7.011227255423254],
 ['u84241937', 4.82781902461732],
 ['u26821983', 8.022367813028454],
 ['u49430339', 4.872828759534886],
 ['u47980817', 8.449561374613236],
 ['u35327419', 3.292781749227846],
 ['u44822085', 6.45450493755737],
 ['u20595103', 4.095924419998536],
 ['u22517588', 8.653203362033977],
 ['u85024818', 0.48542682717024166],
 ['u26552383', 3.786596361890807],
 ['u64894443', 2.2016338611696504],
 ['u46001980', 5

In [13]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [14]:
avgPerItem = {}
for k in reviewsPerItem.keys():
    avgPerItem[k] = sum(h for _,h in reviewsPerItem[k]) / len(reviewsPerItem[k])

In [15]:
avgPerUser = {}
for k in reviewsPerUser.keys():
    avgPerUser[k] = sum(h for _,h in reviewsPerUser[k]) / len(reviewsPerUser[k])

# Simple latent factor-based recommender

In [16]:
# mean hours played
alpha = sum([h for _,_,h in train_data]) / len(train_data)
print(alpha)

3.7160737367374947


In [17]:
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [18]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]

In [19]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

In [20]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(u,g) for u,g,_ in train_data]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

In [21]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(dataset)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for u,g,h in train_data:
        u,i = u,g
        pred = prediction(u, i)
        diff = pred - h
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return np.array(dtheta)

In [23]:
labels = [h for _,_,h in train_data]

In [24]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (labels, 1))

MSE = 5.278092701060167
MSE = 5.159380409631045
MSE = 5.271011677843547
MSE = 5.271101186522287
MSE = 5.271018893684846
MSE = 5.271012318649552
MSE = 5.271011735220723
MSE = 5.2710116829847475
MSE = 5.271011678304242
MSE = 5.271011677884764
MSE = 5.271011677847367
MSE = 5.271011677843823
MSE = 5.271011677843571
MSE = 5.271011677843546
MSE = 5.271011677843546
MSE = 5.271011677843551
MSE = 5.271011677843552
MSE = 5.271011677843551


(array([ 3.71607374e+00, -5.74914752e-05, -9.98419766e-06, ...,
        -7.42451297e-04,  1.44789027e-05, -3.00327606e-04]),
 5.27438849867743,
 {'grad': array([ 1.65065088e-03, -9.63418124e-07, -4.20477381e-08, ...,
         -1.53626750e-05,  2.53377022e-07, -5.99535391e-06]),
  'task': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
  'funcalls': 18,
  'nit': 2,
  'warnflag': 0})

In [25]:
# Use gradient descent values for validation set predictions and report validation mse
timePlayedPred = [prediction(u, g) for u, g,_ in val_data]

In [26]:
y_val = [h for _,_,h in val_data]

In [27]:
print("MSE:\t", MSE(timePlayedPred, y_val))

MSE:	 5.308258644158534


# Complete latent factor-based recommender

In [310]:
# Latent factor model params
userBiases = defaultdict(float)
itemBiases = defaultdict(float)
userGamma = {}
itemGamma = {}
K = 3

In [311]:
for u in reviewsPerUser:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [312]:
for i in reviewsPerItem:
    itemGamma[i] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [313]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    alpha = theta[index]
    index += 1
    userBiases = dict(zip(users, theta[index:index+nUsers]))
    index += nUsers
    itemBiases = dict(zip(items, theta[index:index+nItems]))
    index += nItems
    for u in users:
        userGamma[u] = theta[index:index+K]
        index += K
    for i in items:
        itemGamma[i] = theta[index:index+K]
        index += K

In [314]:
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])

In [315]:
def prediction(user, item):
    #if (user not in userBiases) or (item not in itemBiases):
    #    return alpha
    return alpha + userBiases[user] + itemBiases[item] + inner(userGamma[user], itemGamma[item])

In [316]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(u,g) for u,g,_ in train_data]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in users:
        cost += lamb*userBiases[u]**2
        for k in range(K):
            cost += lamb*userGamma[u][k]**2
    for i in items:
        cost += lamb*itemBiases[i]**2
        for k in range(K):
            cost += lamb*itemGamma[i][k]**2
    return cost

In [317]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(train_data)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in reviewsPerUser:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in reviewsPerItem:
        dItemGamma[i] = [0.0 for k in range(K)]
    for u,g,h in train_data:
        u,i = u,g
        pred = prediction(u, i)
        diff = pred - h
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
        for k in range(K):
            dUserGamma[u][k] += 2/N*itemGamma[i][k]*diff
            dItemGamma[i][k] += 2/N*userGamma[u][k]*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
        for k in range(K):
            dUserGamma[u][k] += 2*lamb*userGamma[u][k]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
        for k in range(K):
            dItemGamma[i][k] += 2*lamb*itemGamma[i][k]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    for u in users:
        dtheta += dUserGamma[u]
    for i in items:
        dtheta += dItemGamma[i]
    return np.array(dtheta)

In [318]:
labels = [h for _,_,h in train_data]

In [331]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + # Initialize alpha
                                   [0.0]*(nUsers+nItems) + # Initialize beta
                                   [random.random() * 0.1 - 0.05 for k in range(K*(nUsers+nItems))], # Gamma
                             derivative, args = (labels, 0.0001))

MSE = 5.364374843164885
MSE = 5.722647809270325
MSE = 5.270013694921394
MSE = 5.262324099622582
MSE = 5.231814425780538
MSE = 5.113755047657899
MSE = 4.705186584885505
MSE = 3.6521667230788655
MSE = 3.388274226051684
MSE = 3.142842085321473
MSE = 3.0438314610399964
MSE = 3.0236715338639537
MSE = 2.9569436105363502
MSE = 2.9199155533941643
MSE = 2.930233657756379
MSE = 2.9107497399683977
MSE = 2.891511738593166
MSE = 2.8848365212780323
MSE = 2.992269390407398
MSE = 2.8848123461177595
MSE = 2.8867840127000104
MSE = 2.8852990407053745
MSE = 2.8795137339932158
MSE = 2.862045388127915
MSE = 2.8160673229425326
MSE = 2.8370690669849457
MSE = 2.814429059095538
MSE = 2.756325826659073
MSE = 2.754096953793116
MSE = 2.7267967192412215
MSE = 2.704565772837458
MSE = 7.285141276849176
MSE = 2.7034749656585566
MSE = 2.6606046467869975
MSE = 2.6614617828426064
MSE = 2.615147544620685
MSE = 2.5768223566173165
MSE = 2.5458748776456135
MSE = 2.5387992217539908
MSE = 2.509593930641651
MSE = 2.494917376461

MSE = 2.3434588809370664
MSE = 2.343427927947094
MSE = 2.3429606509112495
MSE = 2.343253322194289
MSE = 2.3431285636735253
MSE = 2.3430791546415515
MSE = 2.342977147418905
MSE = 2.3427914774190577
MSE = 2.3503258476976914
MSE = 2.342836478000444
MSE = 2.34272142797031
MSE = 2.3426933642966503
MSE = 2.3427031743876903
MSE = 2.3427104812078676
MSE = 2.3426986123874927
MSE = 2.343042089964715
MSE = 2.3427902068316837
MSE = 2.3426305547864192
MSE = 2.3425376946373526
MSE = 2.342480571958386
MSE = 2.3424408486761172
MSE = 2.3423398677643306
MSE = 2.3422134852981213
MSE = 2.3422982137261257
MSE = 2.3394885572790343
MSE = 2.3418438323571182
MSE = 2.342042707055788
MSE = 2.342175414085941
MSE = 2.3422978768359415
MSE = 2.343176849154963
MSE = 2.342474377215184
MSE = 2.3424353985804642
MSE = 2.342289471621859
MSE = 2.3421223516223817
MSE = 2.3420279464385994
MSE = 2.341662589820843
MSE = 2.3417824663352844
MSE = 2.3418356260296616
MSE = 2.3418115984482455
MSE = 2.342035890412593
MSE = 2.3416951

MSE = 2.3384367938144623
MSE = 2.3381150040335106
MSE = 2.3380437530309313
MSE = 2.3380498560543566
MSE = 2.338070098257153
MSE = 2.338042059341604
MSE = 2.337899010801106
MSE = 2.3355168351470947
MSE = 2.3375426798826626
MSE = 2.337520792075022
MSE = 2.33712658141975
MSE = 2.337469130579105
MSE = 2.3375534374432774
MSE = 2.3373092823084214
MSE = 2.3376400110641664
MSE = 2.3375058436695477
MSE = 2.3375119276329426
MSE = 2.3377486743654767
MSE = 2.337671540791256
MSE = 2.337715103572579
MSE = 2.337892335009058
MSE = 2.3340147269107687
MSE = 2.337731835011236
MSE = 2.337985261665171
MSE = 2.338278506365887
MSE = 2.338130372104299
MSE = 2.338167508728437
MSE = 2.3382384711982684
MSE = 2.338329205195087
MSE = 2.338234724757515
MSE = 2.338098775076364
MSE = 2.3356570905242826
MSE = 2.3379643260097343
MSE = 2.3377159305782746
MSE = 2.3370410005070377
MSE = 2.33729606728568
MSE = 2.3373613068702084
MSE = 2.3373737655869404
MSE = 2.3373559979490452
MSE = 2.3373308030141553
MSE = 2.337031986910

MSE = 2.3333505367437333
MSE = 2.3338601538410395
MSE = 2.333963733692249
MSE = 2.3339784421521856
MSE = 2.333922173096247
MSE = 2.338912884853494
MSE = 2.3339818209953274
MSE = 2.3339524583987648
MSE = 2.3340970768772014
MSE = 2.334166411504396
MSE = 2.336843316280131
MSE = 2.3343338054697926
MSE = 2.334409594997116
MSE = 2.3344355872096823
MSE = 2.3344964809305453
MSE = 2.3343155634969572
MSE = 2.3343611728633378
MSE = 2.3343424290733874
MSE = 2.333952299829799
MSE = 2.3342918315954204
MSE = 2.3339436280718306
MSE = 2.33376175684131
MSE = 2.333693515087679
MSE = 2.333667184905675
MSE = 2.333545450367937
MSE = 2.3333805369987166
MSE = 2.3331329077416325
MSE = 2.332791968754654
MSE = 2.3329081915666827
MSE = 2.3327686933343967
MSE = 2.3328835175200426
MSE = 2.3329985707850756
MSE = 2.332982305411495
MSE = 2.3327609771019606
MSE = 2.3327922583499383
MSE = 2.332770640819603
MSE = 2.3325427617366317
MSE = 2.332515222513809
MSE = 2.3325706066788783
MSE = 2.3324536379485368
MSE = 2.33250891

MSE = 2.3304803983137212
MSE = 2.330507754598333
MSE = 2.3304933365836886
MSE = 2.3304955930686626
MSE = 2.3304944086347925
MSE = 2.330527896461301
MSE = 2.33054770935213
MSE = 2.3306319926135095
MSE = 2.330580915995733
MSE = 2.3305739012716855
MSE = 2.3305281895713543
MSE = 2.330444027658196
MSE = 2.3302190000443344
MSE = 2.3303565039595933
MSE = 2.3303891339401743
MSE = 2.3304190917638787
MSE = 2.3304080608473288
MSE = 2.330381484788687
MSE = 2.3303627184917333
MSE = 2.330329176645334
MSE = 2.3303841606584856
MSE = 2.3303548898792616
MSE = 2.3303298900437164
MSE = 2.330321656904219
MSE = 2.330356506888098
MSE = 2.332258301717025
MSE = 2.3303905403033327
MSE = 2.33045209109627
MSE = 2.330445700720762
MSE = 2.330448594986724
MSE = 2.330455149046878
MSE = 2.3302085903187053
MSE = 2.33018114557369
MSE = 2.330159501801235
MSE = 2.3301603565666364
MSE = 2.3310515003491195
MSE = 2.3301732906407153
MSE = 2.330198097059119
MSE = 2.330231076488097
MSE = 2.3303918908176913
MSE = 2.3303592977806

MSE = 2.3291638857839154
MSE = 2.329192399028571
MSE = 2.3291251533308612
MSE = 2.329186276827139
MSE = 2.329196228537478
MSE = 2.3291948772282556
MSE = 2.3291906246838603
MSE = 2.3291832781435575
MSE = 2.329176022381095
MSE = 2.3291856194526117
MSE = 2.3291834482155633
MSE = 2.329188093826569
MSE = 2.329096892757696
MSE = 2.3291763157445837
MSE = 2.3291922510165133
MSE = 2.329201505573821
MSE = 2.33026757708592
MSE = 2.3292134072236776
MSE = 2.329214197430994
MSE = 2.3292420290093703
MSE = 2.329221356797224
MSE = 2.3292157872406825
MSE = 2.329212509975182
MSE = 2.329157621258168
MSE = 2.3292044000672805
MSE = 2.329199100286014
MSE = 2.3292546893920076
MSE = 2.3292236811692244
MSE = 2.3292260494577217
MSE = 2.329216752859102
MSE = 2.3292142635354627
MSE = 2.3291658286833172
MSE = 2.3292006456009338
MSE = 2.3291972892992243
MSE = 2.329173151170601
MSE = 2.329184546828573
MSE = 2.329180587469567
MSE = 2.3291770584202336
MSE = 2.329174061971527
MSE = 2.3291685070527683
MSE = 2.32917557712

(array([ 3.1789551 ,  0.1552413 , -0.25682367, ..., -0.03715738,
         0.24299902, -0.26115153]),
 2.9032810824912945,
 {'grad': array([-7.39797390e-05, -1.05597323e-07, -2.18434519e-08, ...,
          2.06549138e-09,  1.05599671e-07, -1.96740383e-07]),
  'task': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
  'funcalls': 1763,
  'nit': 1496,
  'warnflag': 0})

In [335]:
# Use gradient descent values for validation set predictions and report validation mse
timePlayedPred = [prediction(u, g) for u, g,_ in val_data]

In [336]:
y_val = [h for _,_,h in val_data]

In [337]:
print("MSE:\t", MSE(timePlayedPred, y_val))

MSE:	 3.2996948586750077


In [126]:
predictions = open("../submissions/hours_played_predict.csv", 'w')
for l in open("../data/pairs_Hours.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,g = l.strip().split('-')
  predictions.write(u + '-' + g + ',' + str(prediction(u, g)) + '\n')

predictions.close()

# Collaborative filtering

In [ ]:
hu1 

In [274]:
def Cosine(u1, u2):
    numer = []
    denom_u1 = []
    denom_u2 = []
    # sum across all items for items in both user's sets
    itemIntersect = itemsPerUser[u1].intersection(itemsPerUser[u2])
    if itemIntersect == set():
        return 0
    for item in itemIntersect:
        # hours played of item for each user
        hu1 = [lst[1] for i, lst in enumerate(reviewsPerUser[u1]) if item in lst][0]
        hu2 = [lst[1] for i, lst in enumerate(reviewsPerUser[u2]) if item in lst][0]
        numer.append(hu1*hu2)
        denom_u1.append(hu1**2)
        denom_u2.append(hu2**2)
    
    try:
        sim = (sum(numer)/np.sqrt((sum(denom_u1)*sum(denom_u2))))
    except ZeroDivisionError:
        print(u1, u2)
        print(itemIntersect)
        print(item)
        print()
        print(denom_u1)
        print(sum(denom_u1))
        print(denom_u2)
        print(sum(denom_u2))
        
    return sim

In [275]:
def predictRatingUser(user,item):
    '''
    Predicts the rating for user (u)'s rating for an item i by using a weighted combination of the weighted ratings for
    the same product by other users, weighted by the other user's similarity to the current user
    gave all other items j, weighted by that items similarity to the current item i.
    '''
    ratings = []
    similarities = []
   
    # For all the reviews made about that item
    for u2,h in reviewsPerItem[item]:
        # User id for the reviews of the item
        #u2 = d['user_id']
       
        # if the user id is the same as the user, don't include it
        if u2 == user: continue
           
        # For the rest, append the time played to the list of all ratings
        ratings.append(h - avgPerUser[u2])
       
        # Calcaulate the (cosine) similarity of that user to the user in question
        # and append to the list of all similarities
        similarities.append(Cosine(user, u2))
       
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return sum(weightedRatings) / sum(similarities)
    # Default behavior
    else:
        # User hasn't rated any similar items
        return avgPerItem[item]

In [276]:
# Prediction using user-to-user similarity above
userPredictions = [predictRatingUser(u, g) for u, g,_ in val_data]

<ipython-input-274-bc665f5cc227>:18: RuntimeWarning: invalid value encountered in double_scalars
  sim = (sum(numer)/np.sqrt((sum(denom_u1)*sum(denom_u2))))


In [277]:
y_val = [h for _,_,h in val_data]

In [278]:
print("MSE:\t", MSE(userPredictions, y_val))

MSE:	 17.036597606607522
